### Issues:
- The missing data are being filled in wrong. The NUFFT will pull missing regions down to 1, but we should really use PCA to patch the missing data. So, iteratively: low-pass filter, PCA patch the previously missing data, iterate.
- Lowpass filter is whack and could be checked against Barnett/DFM
- The LSF housekeeping data is super hacky - do we need something better?

In [ ]:
import os
os.environ['APOGEE_CACHE_PATH'] = "/mnt/ceph/users/apricewhelan/apogee"

import sys
import pathlib
_path = str(pathlib.Path('../').resolve())
if _path not in sys.path:
    sys.path.append(_path)

import corner
from astropy.io import fits
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm.auto import tqdm

from jax.config import config
config.update("jax_enable_x64", True)
import jax
import jax.numpy as jnp

from joaquin.data import 
from joaquin.features import default_phot_names as phot_names
from joaquin

In [ ]:
cache_path = pathlib.Path('../cache').resolve()
cache_path.mkdir(exist_ok=True)

In [ ]:
allstar = at.Table.read('/mnt/home/apricewhelan/data/APOGEE_DR17/allStar-dr17-turbo20-beta.fits', 
                        hdu=1)
# allstar = at.Table.read('/mnt/home/apricewhelan/data/APOGEE_DR16/allStar-r12-gaiaedr3.fits')

wise = at.Table.read('/mnt/home/apricewhelan/data/APOGEE_DR17/APOGEE-DR17-wise-result.fits.gz')
wise.rename_column('apogee_id', 'APOGEE_ID')

allstar['APOGEE_ID'] = allstar['APOGEE_ID'].astype(str)
wise['APOGEE_ID'] = [x.strip() for x in wise['APOGEE_ID'].astype(str)]

In [ ]:
allstar = at.join(allstar, wise, keys='APOGEE_ID', join_type='left')
allstar = at.unique(allstar, keys='APOGEE_ID')

In [ ]:
phot_mask = np.ones(len(allstar), dtype=bool)
for name in phot_names:
    phot_mask &= np.isfinite(allstar[name]) & (allstar[name] > 0)

allstar['TELESCOPE'] = np.array([x.strip() for x in allstar['TELESCOPE']])
stars = allstar[(allstar['LOGG'] < 2.2) & 
                (allstar['LOGG'] > 1.5) &
                (allstar['TEFF'] > 3500) &
                (allstar['TEFF'] < 5000) &
                (allstar['SNR'] > 100) &
                np.isin(allstar['TELESCOPE'], ['apo25m', 'lco25m']) & 
                phot_mask][::10]
len(stars)

In [ ]:
# # for star in tqdm(stars[np.argsort(stars['H'])]):

# X = None
# del_idx = []
# for i, star in enumerate(tqdm(stars)):
#     star_hdul = get_aspcapstar(star)
#     lsf_hdul = get_lsf(star)
    
#     lsf_f = get_lsf_features(lsf_hdul)
#     phot_f = get_phot_features(star)
#     try:
#         spec_f = get_spec_features(star_hdul)
#     except:
#         print(f"failed for {i}")
#         del_idx.append(i)
#         star_hdul.close()
#         lsf_hdul.close()
#         continue
    
#     if X is None:
#         X = np.full((len(stars), len(lsf_f) + len(phot_f) + len(spec_f)), 
#                     np.nan)
        
#         lsf_idx = np.arange(len(lsf_f), dtype=int)
#         phot_idx = np.arange(len(lsf_f), len(lsf_f) + len(phot_f), dtype=int)
#         spec_idx = np.arange(len(lsf_f) + len(phot_f), X.shape[1], dtype=int)
        
#     X[i] = np.concatenate((lsf_f, phot_f, spec_f))
    
#     star_hdul.close()
#     lsf_hdul.close()
    
# #     plt.figure(figsize=(15, 5))
# #     plt.plot(star_hdul[1].data, marker='', drawstyle='steps-mid')
# #     plt.plot(star_hdul[2].data, marker='', drawstyle='steps-mid')

# #     plt.figure(figsize=(15, 5))
# #     plt.plot(spec_f, marker='', drawstyle='steps-mid')

# X = np.delete(X, del_idx, axis=0)

In [ ]:
# y = stars['GAIAEDR3_PARALLAX']
# yerr = stars['GAIAEDR3_PARALLAX_ERROR']
# y = np.delete(y, del_idx)
# y_ivar = 1 / np.delete(yerr, del_idx)**2

In [ ]:
dm = DesignMatrix(stars)
# dm.get_sub_Xy()

In [ ]:
plt.hist(dm.stars['GAIAEDR3_PARALLAX'], 
         bins=np.linspace(-0.5, 2, 128));
plt.yscale('log')

TODO: also color by mean fiber number, etc.

In [ ]:
fig = corner.corner(dm.get_sub_Xy(['lsf'])[0][dm.stars['TELESCOPE'] == 'apo25m'], 
                    plot_density=False, plot_contours=False, color='tab:blue')
_ = corner.corner(dm.get_sub_Xy(['lsf'])[0][dm.stars['TELESCOPE'] == 'lco25m'], 
                  plot_density=False, plot_contours=False, color='tab:orange', fig=fig)

for ax in fig.axes:
    ax.relim()
    ax.autoscale()
    
fig.set_facecolor('w')

In [ ]:
colors = [
    ('GAIAEDR3_PHOT_G_MEAN_MAG', 'J'),
    ('GAIAEDR3_PHOT_BP_MEAN_MAG', 'GAIAEDR3_PHOT_RP_MEAN_MAG'),
    ('J', 'K'),
    ('w1mpro', 'w2mpro'),
    ('w3mpro', 'w4mpro')
]

color_X = np.zeros((X.shape[0], len(colors)))
for i, (p1, p2) in enumerate(colors):
    color_X[:, i] = (X[:, phot_idx[phot_names.index(p1)]] -
                     X[:, phot_idx[phot_names.index(p2)]])
    
_ = corner.corner(color_X, plot_density=False, plot_contours=False)

---

Optimizing the model

In [ ]:
joa = Joaquin(*dm.get_sub_Xy(), 
              L2_slice=dm.idx_map['spec'],
              frozen={'L2_ivar': 1e-1, 'parallax_zpt': -0.03})  # MAGIC NUMBER

In [ ]:
beta = joa.init_beta()

In [ ]:
res = joa.optimize()

In [ ]:
res

In [ ]:
fit_pars = joa.unpack_pars(res.x)

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(fit_pars['beta'][dm.idx_map['spec']])
# plt.xlim(100, 1500)

In [ ]:
# beta = fit_pars['beta']
beta = joa.init_beta(L2_ivar=1e2)
pred_plx = np.exp(np.dot(dm._X, beta))

In [ ]:
bins = np.linspace(-0.5, 2, 128)
plt.hist(dm.stars['GAIAEDR3_PARALLAX'], 
         bins=bins);
plt.hist(pred_plx, bins=bins)
plt.yscale('log')

In [ ]:
beta = joa.init_beta(L2_ivar=0.5)

In [ ]:
p0 = [0., 0.5] + list(beta)
joa(p0)

In [ ]:
# test = jax.value_and_grad(joa.__call__)
obj = jax.value_and_grad(neg_ln_posterior, argnums=[3, 4, 5])
def wrapper(*args, **kwargs):
    val, grads = obj(*args, **kwargs)
    return val, jnp.concatenate([g.reshape(-1) for g in grads])

In [ ]:
# test(p0)
val, grad = wrapper(joa.X, joa.y, joa.y_ivar, 
                    0., 0.5, beta, joa.L2_slice)

In [ ]:
grad

In [ ]:
grad

In [ ]:
jnp.dot(np.random.random(size=(10, 3)))

In [ ]:
plt.plot(beta[dm.idx_map['lsf']])

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(phot_names, beta[dm.idx_map['phot']])
plt.xticks(rotation=45, ha='right')

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(beta[dm.idx_map['spec']])
plt.xlim(800, 1200)

### Old plots:

In [ ]:
pix = np.arange(8575, dtype='f8')
wvln = 10 ** (star_hdul[1].header['CRVAL1'] +
              np.arange(star_hdul[1].header['NAXIS1']) * star_hdul[1].header['CDELT1'])
ln_wvln = np.log(wvln)
flux = star_hdul[1].data
err = star_hdul[2].data

mask = (flux == 0 ) | (err > (3 * np.median(err)))

plt.figure(figsize=(15, 5))
plt.plot(wvln[~mask], flux[~mask], marker='', drawstyle='steps-mid')
# plt.plot(wvln[mask], flux[mask], marker='o', ls='none', color='r')

In [ ]:
new_flux = nufft_lowpass(ln_wvln, flux, 
                         fcut=0.5 * 22500, bad_mask=mask)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(wvln, flux, marker='', drawstyle='steps-mid')
plt.plot(wvln, new_flux, 
         marker='', drawstyle='steps-mid', color='tab:blue')
plt.plot(wvln[mask], flux[mask], 
         marker='.', ls='none', color='r')
plt.xlim(15500+500, 15600+500)
plt.axhline(1.)

---

In [ ]:
for star in stars[:4]:
    star_hdul = get_aspcapstar(star)
    lsf_hdul = get_lsf(star)
    
    plt.figure(figsize=(15, 4))
    plt.plot(lsf_hdul[0].data[:, 7], 
             marker='', drawstyle='steps-mid', alpha=0.5)